<a href="https://colab.research.google.com/github/ajaycode/machine-learning/blob/main/Content%20Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content summarization

reference: https://datasciencecastnet.home.blog/2022/12/30/distilhn-summarizing-news-articles-with-transformers/

source: https://colab.research.google.com/drive/1WiCszznFmwdlrsozagT6IJSnuOLQUFqK?usp=sharing#scrollTo=da45ccf3

In [3]:
!pip install -qq feedparser trafilatura feedgenerator

     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 1.0 MB 72.8 MB/s 
     |████████████████████████████████| 837 kB 66.0 MB/s 
     |████████████████████████████████| 140 kB 76.1 MB/s 
     |████████████████████████████████| 412 kB 68.6 MB/s 
     |████████████████████████████████| 293 kB 80.2 MB/s 
     |████████████████████████████████| 195 kB 60.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
aiohttp 3.8.3 requires charset-normalizer<3.0,>=2.0, but you have charset-normalizer 3.0.1 which is incompatible.


In [5]:
!pip install trafilatura

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import trafilatura
import feedparser
import requests
from bs4 import BeautifulSoup
from feedgenerator import DefaultFeed

## Getting Text from a URL

In [7]:
# Download an example website
downloaded = trafilatura.fetch_url('https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/')

In [8]:
# Extract the main text (And show the start)
trafilatura.extract(downloaded, include_comments=False, include_tables=False)[:2048]

'Every March we recognize the women who have shaped history—and now, we’re taking a look forward. From driving software development in large companies to maintaining thriving open source communities, we’re spending Women’s History Month with women leaders who are making history every day in the tech community. Erin Spiceland is a Software Engineer for SpaceX. Born and raised in rural south Georgia, she is a Choctaw and Chickasaw mother of two now living in downtown Los Angeles. Erin didn’t finish college—she’s a predominantly self-taught software engineer. In her spare time, she makes handmade Native American beadwork and regalia and attends powwows.\nHow would you summarize your career (so far) in a single sentence?\nMy career has been a winding road through periods of stimulation and health as well as periods of personal misery. During it all, I’ve learned a variety of programming languages and technologies while working on a diverse array of products and services. I’m a domestic abu

In [9]:
# Make a function to do this
def get_text(url):
    downloaded = trafilatura.fetch_url(url)
    return trafilatura.extract(downloaded, include_comments=False, include_tables=False)

## Sumarization

See https://huggingface.co/inference-api and get your token from https://huggingface.co/settings/tokens (or visit the model page at https://huggingface.co/facebook/bart-large-cnn, click 'Deploy', choose Inference API and click 'show token to get the code ready to go)

In [ ]:
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
headers = {"Authorization": "Bearer YOUR_API_KEY"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [ ]:
def summarize(text):
    if text is None: return None
    output = query({
        "inputs": text[:2048],
            "max_length":300, 
            "min_length":30, 
            "do_sample":False
    })
    return output[0]['summary_text']

In [ ]:
# Test this:
text = get_text('https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/')
summarize(text)

## Getting and Summarizing the feed

In [ ]:
HN_Feed = feedparser.parse('https://hnrss.org/frontpage')

In [ ]:
len(HN_Feed.entries)

In [ ]:
# Run through the first 3 entries, getting the text and summarizing:
for p in HN_Feed.entries[:3]:
    summary = p['summary']
    if 'Article URL' in summary:
        soup = BeautifulSoup(summary, 'html.parser')
        url = soup.find('a').get('href')
        text = get_text(url)
        if text is not None:
            summary = summarize(text)
    print(f"Title: {p['title']}")
    print(f"Summary: {summary}\n\n")

# Sharing with the World

I use the `feedgenerator` package to write the summaries to a new feed (with some custom extra steps for sites like YouTube and Twitter). You can load this in your favourite feed reader by pointing it at https://www.distilhn.com/feed/feed.xml. I also made a basi index.html file that uses javascript to read this feed and insert each item into the page, visible at https://www.distilhn.com. feel free to view the page source to see how that works :)